# Code Conversion using Ollama Inference Models

In [1]:
from itertools import permutations
from constants import *

import ollama
import os
import re

In [2]:
RESULTS_FOLDER = "results"
GENERATED_CODE_FOLDER = os.path.join(RESULTS_FOLDER, "generated_code")
REJECTED_CODE_FOLDER = os.path.join(RESULTS_FOLDER, "rejected_code")
MODEL = "magicoder7b"
LANG2LANG = list(permutations(LANGS.keys(), 2))

In [3]:
def get_prompt(system_prompt, user_prompt):
    prompt = []
    if system_prompt:
        prompt.append({"role": "system", "content": system_prompt})
    prompt.append({"role": "user", "content": user_prompt})
    return prompt

In [4]:
prompt = "Convert the following code from {} to {}. This is the requirement for the code - {}\n```{}```\n"

In [5]:
ollama.pull(OLLAMA_TAGS[MODEL])

{'status': 'success'}

In [6]:
for TASK in TASKS.keys():
    for idx, (lang1, lang2) in enumerate(LANG2LANG):
        if REFERENCE_CODE[lang1][TASK] == "":
            continue
        if os.path.exists(
            os.path.join(
                GENERATED_CODE_FOLDER,
                f"{MODEL}-{TASK}-{lang1.lower()}-{lang2.lower()}.md",
            )
        ):
            continue
        if os.path.exists(
            os.path.join(
                REJECTED_CODE_FOLDER,
                f"{MODEL}-{TASK}-{lang1.lower()}-{lang2.lower()}.md",
            )
        ):
            continue
        formatted_prompt = prompt.format(
            LANGS[lang1],
            LANGS[lang2],
            TASK_DESCRIPTION[TASK],
            REFERENCE_CODE[lang1][TASK],
        )
        if OLLAMA_SYSTEM_PROMPT_SUPPORT[MODEL]:
            chat_prompt = get_prompt(
                "You are a helpful code conversion assistant.", formatted_prompt
            )
        else:
            chat_prompt = get_prompt(None, formatted_prompt)
        try:
            rsp = ollama.chat(
                OLLAMA_TAGS[MODEL],
                messages=chat_prompt,
                options={"seed": 42, "timeout": 60},
            )
        except:
            try:
                rsp = ollama.chat(
                    OLLAMA_TAGS[MODEL],
                    messages=chat_prompt,
                    options={"seed": 41, "timeout": 60},
                )
            except:
                print(f"timeout in {MODEL} {TASK} {lang1} {lang2}")
                with open(
                    os.path.join(
                        REJECTED_CODE_FOLDER,
                        f"{MODEL}-{TASK}-{lang1.lower()}-{lang2.lower()}.md",
                    ),
                    "w",
                ) as fp:
                    fp.write("\n")
                continue
        with open(
            os.path.join(
                GENERATED_CODE_FOLDER,
                f"{MODEL}-{TASK}-{lang1.lower()}-{lang2.lower()}.md",
            ),
            "w",
        ) as fp:
            fp.write(rsp["message"]["content"] + "\n")
        print(f"completed {MODEL} {TASK} {lang1} {lang2}")

completed magicoder7b logic_ops C CPP
completed magicoder7b logic_ops C GO
completed magicoder7b logic_ops C JAVA
completed magicoder7b logic_ops C JS
completed magicoder7b logic_ops C PY
completed magicoder7b logic_ops C RS
completed magicoder7b logic_ops CPP C
completed magicoder7b logic_ops CPP GO
completed magicoder7b logic_ops CPP JAVA
completed magicoder7b logic_ops CPP JS
completed magicoder7b logic_ops CPP PY
completed magicoder7b logic_ops CPP RS
completed magicoder7b logic_ops GO C
completed magicoder7b logic_ops GO CPP
completed magicoder7b logic_ops GO JAVA
completed magicoder7b logic_ops GO JS
completed magicoder7b logic_ops GO PY
completed magicoder7b logic_ops GO RS
completed magicoder7b logic_ops JAVA C
completed magicoder7b logic_ops JAVA CPP
completed magicoder7b logic_ops JAVA GO
completed magicoder7b logic_ops JAVA JS
completed magicoder7b logic_ops JAVA PY
completed magicoder7b logic_ops JAVA RS
completed magicoder7b logic_ops JS C
completed magicoder7b logic_ops J

In [7]:
ollama.delete(OLLAMA_TAGS[MODEL])

{'status': 'success'}